# Potential Field Motion Planning

This notebook implements a continuous potential-field approach for motion planning.

The total cost function consists of:
- A quadratic attraction term toward the goal
- Quadratic-band repulsive penalties around wall segments

Motion is generated using gradient descent on the total cost function.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## 2. Projection onto a Line Segment

To compute the wall penalty, we must determine the closest point on a wall segment
to the current robot position.

This function computes that projection.

In [ ]:
def point_to_segment_projection(x, a, b):
    v = b - a
    w = x - a
    vv = np.dot(v, v)
    if vv == 0.0:
        return a
    t = np.dot(w, v) / vv
    t = np.clip(t, 0.0, 1.0)
    return a + t * v


## 3. Goal Attraction Term

The goal is modeled as a quadratic potential well:

J_goal(x) = 0.5 * alpha * ||x - goal||^2

This produces a smooth gradient pointing toward the goal.

In [ ]:
def goal_cost(x, goal, alpha=1.0):
    return 0.5 * alpha * np.sum((x - goal)**2)

def goal_gradient(x, goal, alpha=1.0):
    return alpha * (x - goal)


## 4. Quadratic-Band Wall Penalty

Each wall creates a smooth repulsive region within radius R.

If the distance to the wall is less than R, we apply a quadratic penalty.
Otherwise, the penalty is zero.

Both cost and gradient are computed analytically.

In [ ]:
def wall_cost_and_gradient(x, a, b, R=1.0, beta=10.0):
    q = point_to_segment_projection(x, a, b)
    diff = x - q
    d = np.linalg.norm(diff)
    if d == 0:
        return 0.0, np.zeros(2)
    if d <= R:
        cost = 0.5 * beta * (R - d)**2
        grad = beta * (d - R) * (diff / d)
        return cost, grad
    else:
        return 0.0, np.zeros(2)


## 5. Total Cost Function

The total potential is the sum of:
- Goal attraction
- Repulsion from all walls

We compute both total cost and total gradient.

In [ ]:
def total_cost_and_gradient(x, walls, goal, alpha=1.0, R=1.0, beta=10.0):
    total_cost = 0.0
    total_grad = np.zeros(2)
    total_cost += goal_cost(x, goal, alpha)
    total_grad += goal_gradient(x, goal, alpha)
    for a, b in walls:
        c, g = wall_cost_and_gradient(x, a, b, R, beta)
        total_cost += c
        total_grad += g
    return total_cost, total_grad


## 6. Gradient Descent Simulation

We simulate motion using gradient descent:

x_{k+1} = x_k - step_size * gradient

The simulation stops when:
- The gradient is small
- The goal is reached
- Maximum iterations are reached


In [ ]:
def simulate_motion(start, walls, goal, alpha=1.0, R=1.0, beta=10.0, step_size=0.05, max_iters=1000, tol=1e-3):
    x = start.copy()
    trajectory = [x.copy()]
    costs = []
    for _ in range(max_iters):
        cost, grad = total_cost_and_gradient(x, walls, goal, alpha, R, beta)
        costs.append(cost)
        if np.linalg.norm(grad) < tol:
            break
        x = x - step_size * grad
        trajectory.append(x.copy())
        if np.linalg.norm(x - goal) < tol:
            break
    return np.array(trajectory), np.array(costs)


## 7. Floor Plan Definition

We define a simple floor plan with boundary walls and an interior corridor.
Different layouts can be added for testing parameter sensitivity.

In [ ]:
def load_floor_plan(name="baseline"):
    if name == "baseline":
        walls = [
            ([-5, -5], [5, -5]),
            ([5, -5], [5, 5]),
            ([5, 5], [-5, 5]),
            ([-5, 5], [-5, -5]),
            ([-1, -5], [-1, 2]),
            ([-1, 2], [3, 2]),
            ([3, 2], [3, -2]),
        ]
        start = [-4.0, -4.0]
        goal = [4.0, 4.0]
    walls = [(np.array(a, dtype=float), np.array(b, dtype=float)) for a, b in walls]
    start = np.array(start, dtype=float)
    goal = np.array(goal, dtype=float)
    return walls, start, goal


In [ ]:
walls, start, goal = load_floor_plan("baseline")
traj, costs = simulate_motion(start, walls, goal, beta=20.0)
plt.figure(figsize=(6,6))
for a, b in walls:
    plt.plot([a[0], b[0]], [a[1], b[1]], "k-", linewidth=2)
plt.plot(traj[:,0], traj[:,1], "b-", linewidth=2)
plt.plot(start[0], start[1], "go")
plt.plot(goal[0], goal[1], "ro")
plt.axis("equal")
plt.grid(True)
plt.title("Gradient Descent Trajectory")
plt.show()
